In [1]:
# import the pandas library
import pandas as pd
import numpy as np

score = pd.DataFrame({
'application_id':[123,234,345,456,567,678],
'zest_score':[0.1, 0.2, 0.3, 0.4, 0.3, 0.15]})
score

,application_id,zest_score
0,123,0.10
1,234,0.20
2,345,0.30
3,456,0.40
4,567,0.30
5,678,0.15


In [3]:
    feature = pd.DataFrame(
         {'application_id':['123','234','345','456','567','567','678', '789'],
'DTI': [10, 20, 30, 40, 50, 50, 30, 40],
'tier':['tier1','tier3','tier3','tier2','tier3','tier3',np.nan, 'tier3'],
'state':['PA','HI','CA','AL','CT','CT','NY','CA'],
'application_date':['2021-12-01','2022-02-23','2022-01-03','2022-01-04','2021-12-31','2021-12-31','2022-02-03','2021-12-18']})
feature

,application_id,DTI,tier,state,application_date
0,123,10,tier1,PA,2021-12-01
1,234,20,tier3,HI,2022-02-23
2,345,30,tier3,CA,2022-01-03
3,456,40,tier2,AL,2022-01-04
4,567,50,tier3,CT,2021-12-31
5,567,50,tier3,CT,2021-12-31
6,678,30,NaN,NY,2022-02-03
7,789,40,tier3,CA,2021-12-18


In [ ]:
# Q1
### Clean the two dataframes and combine them together (keep the intersection). The joined dataframe is called app.

In [5]:
score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
application_id    6 non-null int64
zest_score        6 non-null float64
dtypes: float64(1), int64(1)
memory usage: 224.0 bytes


In [6]:
feature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 5 columns):
application_id      8 non-null object
DTI                 8 non-null int64
tier                7 non-null object
state               8 non-null object
application_date    8 non-null object
dtypes: int64(1), object(4)
memory usage: 448.0+ bytes


In [17]:
feature.application_date = pd.to_datetime(feature.application_date)
feature.application_id = feature.application_id.astype(int)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 0 to 7
Data columns (total 5 columns):
application_id      7 non-null int64
DTI                 7 non-null int64
tier                7 non-null object
state               7 non-null object
application_date    7 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 336.0+ bytes


In [18]:
feature.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 0 to 7
Data columns (total 5 columns):
application_id      7 non-null int64
DTI                 7 non-null int64
tier                7 non-null object
state               7 non-null object
application_date    7 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 336.0+ bytes


In [13]:
feature = feature[~feature.tier.isna()]

In [25]:
feature = feature[~feature.application_id.duplicated(keep='first')]
feature

,application_id,DTI,tier,state,application_date
0,123,10,tier1,PA,2021-12-01
1,234,20,tier3,HI,2022-02-23
2,345,30,tier3,CA,2022-01-03
3,456,40,tier2,AL,2022-01-04
4,567,50,tier3,CT,2021-12-31
7,789,40,tier3,CA,2021-12-18


In [19]:
score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
application_id    6 non-null int64
zest_score        6 non-null float64
dtypes: float64(1), int64(1)
memory usage: 224.0 bytes


In [27]:
app = score.merge(feature, on='application_id').sort_values(by=['tier'], ascending=False)
app

,application_id,zest_score,DTI,tier,state,application_date
1,234,0.2,20,tier3,HI,2022-02-23
2,345,0.3,30,tier3,CA,2022-01-03
4,567,0.3,50,tier3,CT,2021-12-31
3,456,0.4,40,tier2,AL,2022-01-04
0,123,0.1,10,tier1,PA,2021-12-01


In [32]:
app.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 1 to 0
Data columns (total 7 columns):
application_id        5 non-null int64
zest_score            5 non-null float64
DTI                   5 non-null int64
tier                  5 non-null object
state                 5 non-null object
application_date      5 non-null datetime64[ns]
application_status    5 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 320.0+ bytes


In [ ]:
# Q2
# An applicant is approved if it satisfies one of the following two criteria:
# 1. DTI < 30 and zest_score < 0.3
# 2. tier >= 2 and zest_score <= 0.2.

In [28]:
app['application_status'] = 0 

In [34]:
app.loc[((app.DTI<30) & (app.zest_score<0.3)) | ((app.tier.isin(['tier2','tier3'])) & (app.zest_score <=0.2)),'application_status'] = 1

In [35]:
app

,application_id,zest_score,DTI,tier,state,application_date,application_status
1,234,0.2,20,tier3,HI,2022-02-23,1
2,345,0.3,30,tier3,CA,2022-01-03,0
4,567,0.3,50,tier3,CT,2021-12-31,0
3,456,0.4,40,tier2,AL,2022-01-04,0
0,123,0.1,10,tier1,PA,2021-12-01,1


In [ ]:
# Q3
# Calculate the application numbers, the average approval rate and the unique number of states for each month, save the result in one dataframe called app_month

In [45]:
app_month = app.groupby([app['application_date'].dt.month]).agg({'application_id':'count','state':'nunique','application_status':'mean'})
app_month

,application_id,state,application_status
application_date,,,
1,2,2,0.0
2,1,1,1.0
12,2,2,0.5
